In [5]:
#after running this, no need to restart the kurnel when you modify other files.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


File loading and Set a view point

In [6]:
import os
import open3d
from settings import DATA_PATH
import clusterization.clusterize as clusterizer
import numpy as np
from utils.visualizer import save_camera_params
from utils.handling_cluster import visualize_by_cluster, crop_cluster
from detection.detect import compute_nearest_cluster, change_cluster_color
import utils.visualizer as visualizer

data_type_name_pcd = "LIVOX_Hallway_pcds"
experiment_name = "jogging_fast_4th"
dir_name = "res100ms_start13.5s"

#directory path of pcd files
directory_path = os.path.join(DATA_PATH, data_type_name_pcd, experiment_name, dir_name)
#'{DATA_PATH}/LIVOX_Hallway_pcds/jogging_fast_4th/res100ms_start13.5s'

#add save_path
experiment_name_clustered = experiment_name + "_clustered"
dir_name_clustered= dir_name + "_clustered"
folder_clustered = os.path.join(DATA_PATH, data_type_name_pcd, experiment_name_clustered, dir_name_clustered)

if not os.path.exists(folder_clustered):
    os.makedirs(folder_clustered)

# make the list of files
file_list = []
for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)):
        file_list.append(os.path.join(directory_path, filename))
        
#select your view point and close your window.
pcd = open3d.io.read_point_cloud(file_list[0])
cam_params = save_camera_params(pcd)


set the view point and close the window


Detection

In [14]:
#detects nearest cluster from previous frame's cluster sequentially
for i, pcd_file in enumerate(file_list):
    pcd = open3d.io.read_point_cloud(pcd_file)
    
    #initialization
    if i==0:
        eps = 0.5 #this needs to be changed according to distance from LiDAR machine
        center_prev = np.zeros(3) #initialize center of human cluster in previous frame
    
    pcd_clustered = folder_clustered + "/" + os.path.basename(pcd_file) #pcd_clustered_save_path   
    
    #clusterization
    pcd, labels = clusterizer.clusterize(pcd, eps=eps,
                             time_duration=None, #time duration that each windows open for. If you choose None, close window manually.
                             window_name=os.path.basename(pcd_file), #window name changes accorrding to file name.
                             cam_params=cam_params,#camera parameter(view point) you selected
                             save_path=pcd_clustered,
                             visualize=False
                             )
    
    #get cluster_id of human cluster
    if i==0: #select human cluster in first frame manually
        visualizer.visualize(pcd, time_duration=None, cam_params=cam_params)
    
        #select cluster id in comand line. check each window name!!!
        visualize_by_cluster(pcd, labels)
        cluster_id = int(input("Please enter cluster id: "))
        
    else: #compute nearest cluster from previous frame's cluster
        cluster_id = compute_nearest_cluster(pcd, labels, center_prev)
        
    #change the color of points in cluster_id into red.
    pcd = change_cluster_color(pcd, labels, cluster_id=cluster_id)
    visualizer.visualize(pcd, time_duration=0.5, cam_params=cam_params)
    
    #crop (and save) selected cluser
    cropped_pcd = crop_cluster(pcd, labels, cluster_id=cluster_id, visualize=False)
    
    
    #compute center and distance between two centers
    center_tmp = cropped_pcd.get_center()
    current_dist = np.linalg.norm (center_prev - center_tmp)
    
    #update center if current cluster is close to previous one
    if current_dist < 3:
        center_prev = center_tmp
    
    
    # save_cluster(pcd, save_path)
    

#label_list could be used later.

Plane equation: 0.00x + -0.01y + 1.00z + -2.29 = 0
Plane equation: -0.00x + -0.02y + 1.00z + 1.04 = 0
num_clusters 11
Plane equation: 0.00x + -0.02y + 1.00z + -2.28 = 0
Plane equation: -0.00x + -0.02y + 1.00z + 1.04 = 0
num_clusters 12
Plane equation: 0.00x + -0.02y + 1.00z + 1.03 = 0
Plane equation: 0.00x + -0.02y + 1.00z + -2.28 = 0
num_clusters 13
Plane equation: -0.00x + -0.02y + 1.00z + 1.04 = 0
Plane equation: 0.00x + -0.02y + 1.00z + -2.28 = 0
num_clusters 12
Plane equation: 0.00x + -0.02y + 1.00z + -2.28 = 0
Plane equation: -0.00x + -0.02y + 1.00z + 1.04 = 0
num_clusters 16
Plane equation: -0.00x + -0.02y + 1.00z + 1.04 = 0
Plane equation: 0.00x + -0.02y + 1.00z + -2.28 = 0
num_clusters 13
Plane equation: -0.00x + -0.02y + 1.00z + 1.04 = 0
Plane equation: 0.00x + -0.02y + 1.00z + -2.28 = 0
num_clusters 12
Plane equation: -0.00x + -0.01y + 1.00z + 1.03 = 0
Plane equation: 0.00x + -0.02y + 1.00z + -2.28 = 0
num_clusters 12
Plane equation: -0.00x + -0.02y + 1.00z + 1.03 = 0
Plane 

In [5]:
center_prev

array([1.82205592, 0.28191469, 0.10410521])